# Create a Simple Reflex-Based Lunar Lander Agent

In this example, we will use Gymnasium, an environment to train agents via reinforcement learning (RL). We will not use RL here but just use the environment with a custom simple reflex-based agent.

In [10]:
import gymnasium as gym
import numpy as np

## The Lunar Lander Environment
The documentation of the environment is available at: https://gymnasium.farama.org/environments/box2d/lunar_lander/

Performance Measure: A reward of -100 or +100 points for crashing or landing safely respectively. We do not use intermediate rewards here.

Environment: This environment is a classic rocket trajectory optimization problem. A ship needs to land safely. The space is continuous with x and y coordinates in the range [-2.5, 2.5]. The landing pad is at coordinate (0,0).

Actuators: According to Pontryagin’s maximum principle, it is optimal to fire the engine at full throttle or turn it off. This is the reason why this environment has discrete actions: engine on or off. There are four discrete actions available:

0: do nothing </br>
1: fire left orientation engine </br>
2: fire main engine </br>
3: fire right orientation engine </br>

Sensors: Each observation is an 8-dimensional vector: the coordinates of the lander in x & y, its linear velocities in x & y, its angle, its angular velocity, and two booleans that represent whether each leg is in contact with the ground or not.

Gymnasim environments are implemented as classes with a make method to create the environment, a reset method, and a step method to execute an action. To use it with an agent function that expects percetps and returns an action, we need write glue code that connects the environment with the agent function.

In [1]:
def run_episode(agent_function, max_steps=1000):
    """Run one episode in the LunarLander-v3 environment using the provided agent."""

    # Initialize the environment
    env = gym.make("LunarLander-v3", render_mode="human")

    # Reset the environment to generate the first observation (use seed=42 in reset to get reproducible results)
    observation, info = env.reset()

    # run one episode
    for _ in range(max_steps):
        # call the agent function to select an action
        action = agent_function(observation)

        print (f"Obs: {observation} -> Action: {action}")

        # step: execute an action in the environment
        observation, reward, terminated, truncated, info = env.step(action)

        env.render()

        if terminated:
            print(f"Final Reward: {reward}")
            break

    env.close()
    return reward

Note: `env.render()` shows the environment when the notebook is locally run (e.g., in VScode). On Colab, you cannot see the environment because the code is run on a headless server (i.e., a server without a display). There are some workarounds you can google.

## Example: A Random Agent

We ranomly return one of the actions. The environment accepts the integers 0-3.

In [2]:
def random_agent_function(observation):
    """A random agent that selects actions uniformly at random. It ignores the observation."""
    return np.random.choice([0, 1, 2, 3], p=[0.25, 0.25, 0.25, 0.25])

run_episode(random_agent_function)

Obs: [ 0.00699482  1.4206815   0.70848763  0.43381736 -0.00809848 -0.16048309
  0.          0.        ] -> Action: 0
Obs: [ 0.01398983  1.429865    0.7075294   0.4080986  -0.01602847 -0.15861328
  0.          0.        ] -> Action: 1
Obs: [ 0.02089577  1.4384515   0.6963295   0.38154587 -0.02170275 -0.11349607
  0.          0.        ] -> Action: 1
Obs: [ 0.02771311  1.4464304   0.6852125   0.35456842 -0.02514782 -0.06890778
  0.          0.        ] -> Action: 3
Obs: [ 0.03460445  1.4538103   0.6944918   0.32789117 -0.03045116 -0.1060765
  0.          0.        ] -> Action: 3
Obs: [ 0.0415842   1.4605864   0.7055878   0.30098766 -0.03797582 -0.150507
  0.          0.        ] -> Action: 2
Obs: [ 0.04873676  1.467983    0.722178    0.3285442  -0.04482257 -0.13694777
  0.          0.        ] -> Action: 0
Obs: [ 0.05588961  1.47478     0.72219676  0.30186215 -0.05166922 -0.13694568
  0.          0.        ] -> Action: 0
Obs: [ 0.06304264  1.4809777   0.7222169   0.2751942  -0.05851455 -

-100

## A Simple Reflex-Based Agent

To make the code easier to read, we use enumerations for actions (integers) and observations (index in the observation vector).

In [ ]:
from enum import Enum

class Act(Enum):
    LEFT = 1
    RIGHT = 3
    MAIN = 2
    NO_OP = 0

class Obs(Enum):
    X = 0
    Y = 1
    VX = 2
    VY = 3
    ANGLE = 4
    ANGULAR_VELOCITY = 5
    LEFT_LEG_CONTACT = 6
    RIGHT_LEG_CONTACT = 7

In [5]:
def rocket_agent_function(observation):
    """A simple agent function."""

    # run the main thruster, if the lander is falling too fast
    if observation[Obs.VY.value] < -.3:
        return Act.MAIN.value

    return Act.NO_OP.value

run_episode(rocket_agent_function)

Obs: [-0.00146637  1.4130855  -0.14854346  0.09623566  0.00170597  0.03364731
  0.          0.        ] -> Action: 0
Obs: [-0.00293293  1.4146729  -0.14834249  0.07055318  0.00336865  0.03325726
  0.          0.        ] -> Action: 0
Obs: [-0.00439959  1.4156607  -0.14834775  0.04390296  0.00503032  0.0332367
  0.          0.        ] -> Action: 0
Obs: [-0.00586624  1.4160485  -0.14835268  0.01723137  0.00669181  0.03323274
  0.          0.        ] -> Action: 0
Obs: [-0.00733299  1.4158365  -0.1483576  -0.00943714  0.00835301  0.03322756
  0.          0.        ] -> Action: 0
Obs: [-0.00879974  1.4150243  -0.14836252 -0.03610564  0.01001397  0.03322237
  0.          0.        ] -> Action: 0
Obs: [-0.01026659  1.4136121  -0.14836743 -0.06277409  0.01167466  0.03321717
  0.          0.        ] -> Action: 0
Obs: [-0.01173344  1.4116     -0.14837232 -0.0894426   0.01333511  0.03321213
  0.          0.        ] -> Action: 0
Obs: [-0.01320038  1.4089879  -0.14837721 -0.11611117  0.01499529

-100

## Evaluating the Agent

Run the agent on 100 problems and report the average reward.

In [6]:
def run_episode_test(agent_function):
    """Run one episode in the LunarLander-v3 environment using the provided agent."""

    # Initialise the environment
    env = gym.make("LunarLander-v3", render_mode=None)

    # Reset the environment to generate the first observation
    observation, info = env.reset()

    # run one episode (max. 1000 steps)
    for _ in range(1000):
        # call the agent to select an action
        action = agent_function(observation)

        # step (transition) through the environment with the action
        observation, reward, terminated, truncated, info = env.step(action)

        if terminated:
            break

    env.close()
    return reward

def run_episodes(agent_function, n=1000):
    """Run multiple episodes with the given agent and return the rewards for each episode."""
    return [run_episode_test(agent_function) for _ in range(n)]

rewards = run_episodes(rocket_agent_function)
print(rewards)

print(f"Average reward: {np.average(rewards)}")
print(f"Success rate: {np.sum(np.array(rewards) == 100)}/{len(rewards)}")

[-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, 100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100

## Implement A Better Reflex-Based Agent

Build a better that uses its right and left thrusters to land the craft (more) safely. Test your agent function using 100 problems.

In [9]:
def better_rocket_agent(observation):
    """
    A better reflex-based agent that uses left/right thrusters to stabilize
    the lander horizontally and the main engine to reduce vertical speed.
    """

    # Extract observation values
    x = observation[Obs.X.value]
    y = observation[Obs.Y.value]
    vx = observation[Obs.VX.value]
    vy = observation[Obs.VY.value]
    angle = observation[Obs.ANGLE.value]
    angular_velocity = observation[Obs.ANGULAR_VELOCITY.value]
    left_contact = observation[Obs.LEFT_LEG_CONTACT.value]
    right_contact = observation[Obs.RIGHT_LEG_CONTACT.value]

    # If either leg touches ground, do nothing
    if left_contact or right_contact:
        return Act.NO_OP.value

    # Stabilize horizontal movement: if lander drifts right, fire LEFT; if drifts left, fire RIGHT
    if x < -0.05:
        return Act.RIGHT.value
    elif x > 0.05:
        return Act.LEFT.value

    # Stabilize angle: if tilting right, fire LEFT thruster; if tilting left, fire RIGHT thruster
    if angle < -0.05:
        return Act.RIGHT.value
    elif angle > 0.05:
        return Act.LEFT.value

    # Reduce vertical speed if falling too fast
    if vy < -0.2:
        return Act.MAIN.value

    # Otherwise, do nothing
    return Act.NO_OP.value


def run_episode_test(agent_function, max_steps=1000):
    env = gym.make("LunarLander-v3", render_mode=None)
    observation, info = env.reset()
    for _ in range(max_steps):
        action = agent_function(observation)
        observation, reward, terminated, truncated, info = env.step(action)
        if terminated:
            break
    env.close()
    return reward

def run_episodes(agent_function, n=100):
    """Run multiple episodes with the given agent and return the rewards for each episode."""
    return [run_episode_test(agent_function) for _ in range(n)]

# Run the better reflex agent on 100 problems
rewards = run_episodes(better_rocket_agent, n=100)
print(f"Rewards: {rewards}")
print(f"Average reward: {np.average(rewards)}")
print(f"Success rate: {np.sum(np.array(rewards) == 100)}/{len(rewards)}")

Rewards: [-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100]
Average reward: -100.0
Success rate: 0/100
